This notebook is used to diff the dependency surface for Linux kernels with different versions and configurations.

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from depsurf import DepKind, DiffResult, VersionGroup
from utils import OUTPUT_PATH, save_pkl
from typing import List
import pandas as pd


def diff(name: str, groups: List[VersionGroup], kinds: List[DepKind]):
    results = DiffResult(
        {
            group: group.diff_pairs(kinds, OUTPUT_PATH / name / group.name)
            for group in groups
        }
    )
    save_pkl(results, name)

    df = pd.DataFrame(
        {
            (group, group.to_str(pair.v1), group.to_str(pair.v2)): {
                (kind, issue): count
                for kind, kind_result in pair_result.iter_kinds()
                for issue, count in kind_result.iter_issues()
            }
            for group, group_result in results.iter_groups()
            for pair, pair_result in group_result.iter_pairs()
        }
    )
    df = df.loc[(df != 0).any(axis=1)]
    return df


# diff("test", [VersionGroup.LTS], [DepKind.FUNC])

In [3]:
diff(
    "src",
    [VersionGroup.LTS, VersionGroup.REGULAR],
    [
        DepKind.FUNC,
        DepKind.STRUCT,
        DepKind.TRACEPOINT,
        DepKind.LSM,
    ],
)

[version_group.py:140] INFO: Diffing LTS
[          btf.py:44 ] INFO: Loading types from /users/szhong/DepSurf/data/dataset/types/4.4.0-21-generic-amd64.json


[          btf.py:44 ] INFO: Loading types from /users/szhong/DepSurf/data/dataset/types/4.15.0-20-generic-amd64.json
[ version_pair.py:97 ] INFO: Saving to /users/szhong/DepSurf/output/src/LTS/4.4_4.15
[          btf.py:44 ] INFO: Loading types from /users/szhong/DepSurf/data/dataset/types/5.4.0-26-generic-amd64.json
[ version_pair.py:97 ] INFO: Saving to /users/szhong/DepSurf/output/src/LTS/4.15_5.4
[          btf.py:44 ] INFO: Loading types from /users/szhong/DepSurf/data/dataset/types/5.15.0-25-generic-amd64.json
[ version_pair.py:97 ] INFO: Saving to /users/szhong/DepSurf/output/src/LTS/5.4_5.15
[          btf.py:44 ] INFO: Loading types from /users/szhong/DepSurf/data/dataset/types/6.8.0-31-generic-amd64.json
[ version_pair.py:97 ] INFO: Saving to /users/szhong/DepSurf/output/src/LTS/5.15_6.8
[version_group.py:140] INFO: Diffing Regular
[          btf.py:44 ] INFO: Loading types from /users/szhong/DepSurf/data/dataset/types/4.8.0-22-generic-amd64.json
[ version_pair.py:97 ] INFO:

LTS                      Regular                                                                                                         
                                  4.4   4.15    5.4   5.15     4.4    4.8   4.10   4.13   4.15   4.18    5.0    5.3    5.4    5.8   5.11   5.13   5.15   5.19    6.2    6.5
                                 4.15    5.4   5.15    6.8     4.8   4.10   4.13   4.15   4.18    5.0    5.3    5.4    5.8   5.11   5.13   5.15   5.19    6.2    6.5    6.8
Function   Old                  36309  42299  48018  53956   36309  38211  39148  40741  42299  44488  45277  46992  48018  51936  54302  53193  53956  55843  56978  59365
           New                  42299  48018  53956  61983   38211  39148  40741  42299  44488  45277  46992  48018  51936  54302  53193  53956  55843  56978  59365  61983
           Added                 8600   8929  10771  12156    2849   1574   2662   2118   3635   1980   2516   1382   6785   3518   1469   1753   3525   2622   3196   3626
           Removed               2610   3210   4833   4129     947    637   1069    560   1446   1191    801    356   2867   1152   2578    990   1638   1487    809   1008
           Changed               1956   1807   2187   2996     768    386    830    534    821    698    510    513    871    876    602    573    936    903   1058    801
           Param added           1115    963   1316   1534     404    221    513    300    410    342    277    446    540    530    454    264    556    449    579    352
           Param removed          800    659    967   1424     292    178    343    206    194    303    210    384    433    485    189    172    486    347    588    302
           Param reordered        375    349    556    555     173     42    183     87    203     84    101     27    138    189    279    101    241    217    148     98
           Param type changed     508    421    572    751     212    107    166    159    216    175    101     28    173    260     71    193    141    253    199    288
           Return type changed    292    380    288    507     114     54    114     59    179    140     75     23    120     91     52     94    127    146    158    151
Struct     Old                   6160   7342   8422   9329    6160   6599   6752   7051   7342   7623   7807   8209   8422   8610   8954   9178   9329   9551   9767  10022
           New                   7342   8422   9329  10474    6599   6752   7051   7342   7623   7807   8209   8422   8610   8954   9178   9329   9551   9767  10022  10474
           Added                 1449   1364   1410   1504     536    227    371    362    393    301    467    273    480    495    323    232    384    337    318    500
           Removed                267    284    503    359      97     74     72     71    112    117     65     60    292    151     99     81    162    121     63     48
           Changed               1116   1130   1308   1519     554    365    608    378    511    542    524    260    678    614    397    437    697    569    585    569
           Field added            800    836    979   1120     406    248    395    259    359    374    386    183    482    443    268    334    507    400    403    405
           Field removed          458    452    523    634     220    101    200    139    156    173    192     98    240    230    144    130    280    187    211    194
           Field type changed     412    384    422    513     143    117    218    109    167    168    125     57    199    169    110    114    203    176    135    167
Tracepoint Old                    502    675    752    818     502    539    559    635    675    683    704    737    752    785    813    805    818    843    871    917
           New                    675    752    818    932     539    559    635    675    683    704    737    752    785    813    805    818    843    871    917    932
           Added                  197     98    102    150      37     28     

In [4]:
diff(
    "config",
    [VersionGroup.ARCH, VersionGroup.FLAVOR],
    [
        DepKind.CONFIG,
        DepKind.FUNC,
        DepKind.STRUCT,
        DepKind.TRACEPOINT,
        DepKind.SYSCALL,
    ],
)

[version_group.py:140] INFO: Diffing Arch
[          btf.py:44 ] INFO: Loading types from /users/szhong/DepSurf/data/dataset/types/5.4.0-26-generic-arm64.json


[ version_pair.py:97 ] INFO: Saving to /users/szhong/DepSurf/output/config/Arch/amd64_arm64
[          btf.py:44 ] INFO: Loading types from /users/szhong/DepSurf/data/dataset/types/5.4.0-26-generic-armhf.json
[ version_pair.py:97 ] INFO: Saving to /users/szhong/DepSurf/output/config/Arch/amd64_armhf
[          btf.py:44 ] INFO: Loading types from /users/szhong/DepSurf/data/dataset/types/5.4.0-26-generic-ppc64el.json
[ version_pair.py:97 ] INFO: Saving to /users/szhong/DepSurf/output/config/Arch/amd64_ppc64el
[          btf.py:44 ] INFO: Loading types from /users/szhong/DepSurf/data/dataset/types/5.4.0-24-generic-riscv64.json
[ version_pair.py:97 ] INFO: Saving to /users/szhong/DepSurf/output/config/Arch/amd64_riscv64
[version_group.py:140] INFO: Diffing Flavor
[          btf.py:44 ] INFO: Loading types from /users/szhong/DepSurf/data/dataset/types/5.4.0-1009-aws-amd64.json
[ version_pair.py:97 ] INFO: Saving to /users/szhong/DepSurf/output/config/Flavor/generic_aws
[          btf.py:44

Arch                         Flavor                         
                                amd64                        generic                         
                                arm64  armhf ppc64el riscv64     aws  azure    gcp lowlatency
Config     Old                   8776   8776    8776    8776    8776   8776   8776       8776
           New                   9594   9607    8098    7626    6413   5306   8586       8779
           Added                 1933   2194     657     480       8     11      0          9
           Removed               1115   1363    1335    1630    2371   3481    190          6
           Changed                 41     85      51      44     252    871    249          9
           Config changed          41     85      51      44     252    871    249          9
Function   Old                  48018  48018   48018   48018   48018  48018  48018      48018
           New                  49362  48845   42826   36586   46523  45483  48149      48034
           Added                 9216  12602    5444    2079     328    992    450         57
           Removed               7872  11775   10636   13511    1823   3527    319         41
           Changed                120    106     137     101       2     10      1          0
           Param added             63     50      79      55       2      9      1          0
           Param removed           63     50      80      58       2     10      1          0
           Param reordered          3      8      12       7       0      2      0          0
           Param type changed      57     55      61      48       1      3      0          0
           Return type changed      9     10      24       5       0      1      0          0
Struct     Old                   8422   8422    8422    8422    8422   8422   8422       8422
           New                   9075   8551    7368    6578    8022   7846   8367       8425
           Added                 1659   1980     570     157      83    257     68          4
           Removed               1006   1851    1624    2001     483    833    123          1
           Changed                 81    154     116      98      19     28     14          5
           Field added             33     42      32      22       6      7      2          2
           Field removed           39     66      50      72      13     20     11          3
           Field type changed      37     89      75      29       1      6      2          0
Tracepoint Old                    752    752     752     752     752    752    752        752
           New                    685    690     648     625     747    739    752        752
           Added                   45     70      25       0       4     26      0          0
           Removed                112    132     129     127       9     39      0          0
Syscall    Old                    333    333     333     333     333    333    333        333
           New                    291    378     347     280     333    333    333        333
           Added                    2     74      23       2       0      0      0          0
           Removed                 44     29       9      55       0      0      0          0